# Imports & funcs

In [1]:
# !pip install fuzzywuzzy transformers

In [2]:
import os
import re

import pandas as pd
import numpy as np
import torch
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
from sklearn.metrics.pairwise import cosine_similarity
from transformers import AutoTokenizer, AutoModel

/usr/local/lib/python3.10/dist-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [3]:
pd.set_option('display.max_colwidth', None)

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
def read_text_file(file_path):
    with open(file_path, 'r') as f:
        return f.read()

In [6]:
def extract_recipes_names(recipes):
    return list(map(lambda x: x.split('📖')[0].strip(), recipes))

In [7]:
def extract_ingridients(recipes):
    ingredients_lists = []

    for recipe in recipes:
        ingredients_part = re.search(r'Ингредиенты:(.*?)🧑🏻‍🍳', recipe, re.DOTALL)
        if not ingredients_part:
            continue
        ingredients_text = ingredients_part.group(1)

        ingredients = [ingredient.strip() for ingredient in ingredients_text.split('-') if ingredient.strip()]

        ingredients = [re.sub(r'\s*-\s*.*', '', ingredient) for ingredient in ingredients]

        upper_case_ingredients = [ingredient for ingredient in ingredients if ingredient[0].isupper()]

        ingredients_lists.append(' '.join(upper_case_ingredients))

    return ingredients_lists

In [8]:
def encode_text(text):
    input_ids = tokenizer.encode(text, add_special_tokens=True, return_tensors='pt', max_length=512, truncation=True)
    with torch.no_grad():
        outputs = model(input_ids)
    return outputs.last_hidden_state.mean(dim=1)

In [9]:
def get_list_of_recipes(dists_values, threshold):
    ans_indices = np.where(dists_values > threshold)[0]
    values = dists_values[ans_indices]
    sorted_indices_desc = ans_indices[np.argsort(-values)]

    if len(sorted_indices_desc) > 0:
        return sorted_indices_desc, True

    return [], False

In [10]:
tokenizer = AutoTokenizer.from_pretrained('cointegrated/rubert-tiny2')
model = AutoModel.from_pretrained('cointegrated/rubert-tiny2')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


# Data analysis

In [11]:
all_recipe_files = sorted(os.listdir("/content/drive/MyDrive/text/recipe/"))

In [12]:
recipes = []
for file in all_recipe_files:
    recipes.append((file.split('.')[0], read_text_file("/content/drive/MyDrive/text/recipe/" + file)))

In [13]:
data = pd.DataFrame(columns=["recipe_id", "text"])

data["recipe_id"] = list(map(lambda x: x[0], recipes))
data["text"] = list(map(lambda x: x[1], recipes))

data["recipe_id"] = data["recipe_id"].astype(int)

In [14]:
data = data.sort_values("recipe_id")
data.reset_index(drop=True, inplace=True)

# Search

По ингридиентам и рецептам

In [15]:
def get_ans_search(query, dist_threshold=85, sim_threshold=0.7, action_type="recipes"):
    recipes = data.text.tolist()

    func = extract_recipes_names if action_type == "recipes" else extract_ingridients

    # 1 step
    useful_info_recipes = func(recipes)

    lev_dists = np.array(list(map(lambda x: fuzz.WRatio(query, x), useful_info_recipes)))

    ans_recipes, flag = get_list_of_recipes(lev_dists, dist_threshold)

    if flag:
        return ans_recipes

    # 2 step
    search_vector = encode_text(query)
    recipe_vectors = torch.stack([encode_text(recipe) for recipe in useful_info_recipes])

    cosine_similarities = cosine_similarity(search_vector, recipe_vectors.reshape(recipe_vectors.shape[0], recipe_vectors.shape[2]))

    ans_recipes, flag = get_list_of_recipes(cosine_similarities[0], sim_threshold)

    return ans_recipes

Тестирование поиска по рецептам

In [16]:
get_ans_search("банановый кекс")

array([251,   3,   7])

In [17]:
data.text.loc[251], data.text.loc[3], data.text.loc[7]

('Банановый кекс в микроволновке 📖 Ингредиенты: - Банан - 2 шт. - Яйцо - 2 шт. - Сливочное масло - 100 г. - Сахар - 100 г. - Мука - 100 г. - Разрыхлитель - 1 ч. л. - Соль - 1 щепотка - Корица - 2 ч. л. - Молоко - 4 ст. л. 🧑🏻\u200d🍳 Приготовление: 1. Бананы очистить от кожуры, нарезать на небольшие кусочки и размять вилкой или толкушкой в пюре. 2. Смешать банановое пюре с яйцами, растопленным сливочным маслом, сахаром и перемешать. 3. Муку, разрыхлитель, щепотку соли, корицу и молоко добавить в банановую смесь. Перемешать до однородности. 4. Распределить смесь по форме пригодной для микроволновки. Указанное количество ингредиентов рассчитано на емкость объемом 1 литр. 5. Запечь в микроволновке на максимальной мощности (850-1100 ватт) 8 минут. Готовый кекс по желанию присыпать сахарной пудрой #банановыйкекс #кекс #десерт',
 'Банановый пирог на песочном тесте 📖 Ингредиенты: - Мука - 300 г. - Масло сливочное - 100 г. - Банан - 3 шт. - Яйцо - 3 шт. - Сметана - 1 ст.л. - Сахар - 1 ст.л. 🧑🏻\u

Тестирование поиска по ингридиентам

In [18]:
get_ans_search("масло сливочное", action_type="ingridients")[:3]

array([  2,   3, 255])

In [19]:
data.text.loc[2], data.text.loc[3], data.text.loc[255]

('Шоколадный фондан 📖 Ингредиенты:⠀ - Шоколад темный - 100 г. - Масло сливочное - 60 г. - Яйцо- 2 шт. - Сахар - 3 ст.л. ⠀ - Мука - 2 ст.л. ⠀ - Какао - 2 ч.л ⠀ - Разрыхлитель - 1 ч.л ⠀ - Соль - 1/4 ч.л ⠀ ⠀ 🧑🏻\u200d🍳 Приготовление:⠀ 1. Шоколад ломаем на кусочки и растапливаем на водяной бане или микроволновой печи короткими импульсами. Растапливаем сливочное масло в микроволновой печи и соединяем с шоколадом. 2. В отдельной емкости соединяемых яйца и сахар, перемешиваем. 3. Вливаем к яйцам с сахаром шоколадно-масляную смесь, добавляем муку, соль, разрыхлитель и перемешиваем. Раскладываем тесто по формочкам. 4. Духовку разогреваем до 180 и ставим формочки на верхний уровень. Выпекаем 7 минут. Готово! #шоколадныйфондан #десерт',
 'Банановый пирог на песочном тесте 📖 Ингредиенты: - Мука - 300 г. - Масло сливочное - 100 г. - Банан - 3 шт. - Яйцо - 3 шт. - Сметана - 1 ст.л. - Сахар - 1 ст.л. 🧑🏻\u200d🍳 Приготовление: 1. Отделить белки от желтков. Желтки смешать с сахарным песком, добавить раст

# RecSys

In [20]:
def get_recommendation_by_fav(recipe_id, dist_threshold=0.5, sim_threshold=0.5):
    recipes = data[data["recipe_id"] != recipe_id].text.tolist()

    anchor_recipe_name = extract_recipes_names(data[data["recipe_id"] == recipe_id].text.tolist())[0]
    recipes_names = extract_recipes_names(recipes)

    lev_dists = np.array(list(map(lambda x: fuzz.WRatio(anchor_recipe_name, x), recipes_names)))

    ans_recipes, _ = get_list_of_recipes(lev_dists, dist_threshold)

    first_step = ans_recipes[:50]

    anchor_vector = encode_text(anchor_recipe_name)
    recipe_vectors = torch.stack([encode_text(recipe) for recipe in recipes_names])

    cosine_similarities = cosine_similarity(anchor_vector, recipe_vectors.reshape(recipe_vectors.shape[0], recipe_vectors.shape[2]))

    ans_recipes, _ = get_list_of_recipes(cosine_similarities[0], sim_threshold)

    second_step = ans_recipes[:50]

    return [item for item in second_step if item in set(first_step)][:9]

In [21]:
predict_recipes = get_recommendation_by_fav(50)

In [22]:
print(f"Рецепт из избранного:\n{data[data['recipe_id'] == 50].text.values[0]}\n\nРекомендуемые рецепты:\n{data[data['recipe_id'] != 50].text[predict_recipes].values}")

Рецепт из избранного:
Сметанный торт 📖 Ингредиенты: - Сметана - 2 стакана - Мука - 3 стакана - Сахар - 3/4 стакана (плюс 1/2 стакана в крем) - Сода - 1/4 ч. л. - Соль - — 1/4 ч. л. - Ванильный сахар — 1/3 ч. л. 🧑🏻‍🍳 Приготовление: 1. Разогреть духовку до 230-240 градусов. Смазать противень растительным маслом. 2. Перемешать в большой миске сметану, сахар и соль. Всыпать муку и соду, замесить тесто. 3. Разделить готовое тесто на 4 равные части. Из каждой части скалкой раскатать круг. Выложить коржи на подготовленный противень. Выпекать в течение 10-15 минут. 4. Тем временем приготовить крем: Взбить сметану с сахаром и ванильным сахаром. 5. Выложить один корж на большое блюдо, смазать кремом, накрыть сверху вторым коржом, смазать его кремом и т.д. 6. Четвертый корж измельчить до консистенции крошек и посыпать им третий корж, обильно смазанный кремом. Украсить торт по желанию и поставить в холодильник на 3-4 часа. #сметанныйторт #торт #десерт

Рекомендуемые рецепты:
['Шоколадный бисквит н